In [1]:
import numpy as np
import pandas as pd 
import pandas_profiling as pp
import plotly.express as px 
import sqlite3
import random
from sklearn.metrics import mean_squared_error

pd.options.display.max_columns = None

## Import des datasets 

In [2]:
df_aeroports = pd.read_parquet("../data/aggregated_data/aeroports.gzip")
df_compagnies = pd.read_parquet("../data/aggregated_data/compagnies.gzip")
df_vols = pd.read_parquet("../data/aggregated_data/vols.gzip")
df_fuel = pd.read_parquet("../data/aggregated_data/prix_fuel.gzip")
df_test = pd.read_parquet("../data/extracted/test_data/vols.gzip")

In [42]:
df_aeroports

,CODE IATA,NOM,LIEU,PAYS,LONGITUDE,LATITUDE,HAUTEUR,PRIX RETARD PREMIERE 20 MINUTES,PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES
0,MCT,Muscat International Airport,Muscat,OM,58.284400939941406,23.593299865722656,48.0,53,3
1,SOU,Southampton Airport,Southampton,GB,-1.3567999601364136,50.95029830932617,44.0,24,5
2,PNH,Phnom Penh International Airport,Phnom Penh,KH,104.84400177001953,11.546600341796875,40.0,33,3
3,BLR,Kempegowda International Airport,Bangalore,IN,77.706299,13.1979,3000.0,70,9
4,FFD,RAF Fairford,Fairford,GB,-1.7900300025900002,51.6822013855,286.0,65,3
...,...,...,...,...,...,...,...,...,...
317,MLE,MalÃ© International Airport,MalÃ©,MV,73.52909851074219,4.191830158233643,6.0,83,5
318,XMN,Xiamen Gaoqi International Airport,Xiamen,CN,118.12799835205078,24.54400062561035,59.0,81,4
319,SUB,Juanda International Airport,Surabaya,ID,112.78700256347656,-7.3798298835754395,9.0,74,9
320,HND,Tokyo Haneda International Airport,Tokyo,JP,139.779999,35.552299,35.0,13,7


In [41]:
df_vols['COMPAGNIE AERIENNE']

0            MAF
1            I6F
2          NVPPA
3          NVPPA
4            THA
           ...  
4332909    SMITH
4332910    SMITH
4332911    SMITH
4332912    SMITH
4332913    SMITH
Name: COMPAGNIE AERIENNE, Length: 4332914, dtype: object

In [4]:
list_compagnies = df_compagnies['COMPAGNIE'].unique().tolist()
list_airports = df_aeroports['CODE IATA'].unique().tolist()

In [5]:
airlines = [random.choice(list_compagnies) for i in range(100)]
airports = [random.choice(list_airports) for i in range(100)]
nb_passagers = list(random.sample(range(0, 2505), 100))

In [6]:
y_true = np.array(random.sample(range(-100, 1898), 100))
y_preds = np.array(random.sample(range(-100, 1898), 100))
prediction = pd.DataFrame({'RETARD': y_preds, 'COMPAGNIES': airlines, 'AEROPORTS': airports,
                          'NOMBRE DE PASSAGERS': nb_passagers, 'RETARD REEL' : y_true})

# METRICS 

On a choisit la **RMSE** : La racine de l'erreur quadratique moyenne ou racine de l'écart quadratique moyen est une mesure fréquemment utilisée des différences entre les valeurs prédites par un modèle ou estimateur et les valeurs observées

- **Formule** : $RMSE = \sqrt{MSE(\theta)} = \sqrt{\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)^{2}}$

- **Interpretation** : plus la valeur de notre RMSE est grande plus notre prédiction (le retard prédit à l'arrivée) est loin de la réalité terrain, du retard à l'arrivée effectif. Une RMSE proche de 0 signifie que notre modèle est proche de la réalité dans ces prédictions. 

On a également choisi une deuxième métrique : 
**Moyenne** : $\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)$

**Objectif** : pénaliser l'écart entre la prédiction et la réalité terrain mais aussi pénaliser la direction de l'erreur, c'est à dire si notre modèle prédit plus de retard que prévu ou à l'inverse moins de retard que prévu. 

In [7]:
airline_list = list(dict.fromkeys(airlines))
rmse = []
mean_error=[]
for idx, airline in enumerate(airline_list): 
    y_true = prediction[prediction['COMPAGNIES'] == airline]['RETARD REEL']
    y_preds =  prediction[prediction['COMPAGNIES'] == airline]['RETARD']
    rmse.append(mean_squared_error(y_true, y_preds, squared=False)) #if squared=True return MSE value
    mean_error.append((y_true - y_preds).mean())

df_metrics = pd.DataFrame({'COMPAGNIE':airline_list, 'RMSE': rmse, 'MEAN ERROR': mean_error})

In [8]:
df_metrics

,COMPAGNIE,RMSE,MEAN ERROR
0,Better Take A Train Airlines,769.663173,-534.600000
1,Corporate Overlord Airways,800.235937,173.777778
2,Ne Va Pas Partout Airlines,738.822938,199.000000
3,Neverland Airlines,851.268230,12.400000
4,IE 6.0 Flights,960.859437,616.285714
5,Bel Air,1138.183013,92.000000
6,Morally Ambiguious Fligthts,555.527677,293.500000
7,Air Piloter Sans Les Mains,852.793384,250.111111
8,Always A Problem Flights,837.388002,-60.444444
9,Try Hard Airlines,750.959312,-97.888889


# KPIs

In [9]:
prediction["CHIFFRE D'AFFAIRE COMPAGNIE"] = prediction['COMPAGNIES'].map(lambda x:\
                                       df_compagnies[df_compagnies['COMPAGNIE'] ==x]['CHIFFRE D AFFAIRE'].values[0])

In [10]:
prediction_avec_retard = prediction[prediction['RETARD']>0].copy()

### Prix du retard

Hypothese prix retard aéroport (centaine d'euros) : 
- après 10min :  la compagnie paye toutes les minutes le prix indiqué dans la colonne "PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES"
- après 20min : la compagnie paye un **supplément** qui est le prix indiqué dans la colonne "PRIX RETARD PREMIERE 20 MINUTES"

In [11]:
def add_cost_20min_delay(df_aeroports, airport):
    twenty_first_min_cost = df_aeroports[
        df_aeroports['CODE IATA'] == airport]['PRIX RETARD PREMIERE 20 MINUTES'].values[0]
    return twenty_first_min_cost

def add_cost_10min_delay(df_aeroports, airport):
    ten_min_delay_cost = df_aeroports[
            df_aeroports['CODE IATA'] == airport]['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'].values[0]
    return ten_min_delay_cost

In [12]:
def cost_of_delay(pred_vol):
    delay = pred_vol['RETARD']
    twenty_first_min_cost = pred_vol['PRIX RETARD PREMIERE 20 MINUTES']
    ten_min_delay_cost = pred_vol['PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES']
    
    cost = 0
    if delay > 10 : 
        cost += ten_min_delay_cost * (delay - 10) 
    if delay >= 20 : 
        cost += twenty_first_min_cost
    return cost

In [13]:
prediction_avec_retard['PRIX RETARD PREMIERE 20 MINUTES'] = prediction_avec_retard['AEROPORTS']\
                                                            .map(lambda x: add_cost_20min_delay(df_aeroports, x))
    
prediction_avec_retard['PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES'] = prediction_avec_retard['AEROPORTS']\
                                                                .map(lambda x: add_cost_10min_delay(df_aeroports, x))
prediction_avec_retard['COUT DU RETARD'] = prediction_avec_retard.apply(cost_of_delay, axis=1)

In [14]:
prediction_avec_retard = prediction_avec_retard.drop(
    columns=['PRIX RETARD PREMIERE 20 MINUTES', 'PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES'])

### Indemnisation des clients 

Hypothèse : 
- 10% des clients vont demander à être indemnisé pour un retard compris entre 10min et 45min
    - Indemnité à payer : 1/4 du prix du billet
- 20% des clients vont demander à être indemnisé pour un retard supérieur à 1h 
    - Indemnité à payer : 1/2 du prix du billet
- 50% des clients vont demander à être indemnisé pour un retard supérieur à 3h 
    - Indemnité à payer : totalité du prix du billet
    
On fait l'hypothèse d'un fixe maximal du prix du billet : **300€**

In [15]:
def get_number_of_indemnities_asked(pred_vol): 
    delay = pred_vol.loc['RETARD']
    nb_of_passenger = pred_vol.loc['NOMBRE DE PASSAGERS']
    nb_of_indemnities_asked = 0
    if delay > 10 and delay <45: 
        nb_of_indemnities_asked = 20*nb_of_passenger//100
    elif delay > 60 and delay <180:
        nb_of_indemnities_asked = 50*nb_of_passenger//100
    elif delay > 180:
        nb_of_indemnities_asked = 75*nb_of_passenger//100
    return nb_of_indemnities_asked

def compensation_due(pred_vol, ticket_price=300): 
    delay = pred_vol.loc['RETARD']
    nb_of_indemnities_asked = pred_vol.loc["NOMBRE D'INDEMNITES DEMANDEES"]
    compensation_due_to_clients = 0
    if delay > 10 and delay <45: 
        compensation_due_to_clients = (ticket_price/3)*nb_of_indemnities_asked
    elif delay > 60 and delay <180:
        compensation_due_to_clients = (ticket_price/2)*nb_of_indemnities_asked
    elif delay > 180:
        compensation_due_to_clients = ticket_price*nb_of_indemnities_asked
    return compensation_due_to_clients

In [16]:
prediction_avec_retard[
    "NOMBRE D'INDEMNITES DEMANDEES"] = prediction_avec_retard.apply(get_number_of_indemnities_asked, axis=1)
prediction_avec_retard[
    "INDEMNITES A PAYER"] = prediction_avec_retard.apply(compensation_due, axis=1)

## Perte de client : 

**Hypothèse** : Taux d'attrition à 3% pour un retard de plus de 3h 

In [17]:
def get_number_of_lost_customer(delay, passenger_nb):
    if delay > 180 : 
        return passenger_nb*3//100
    else : return 0

prediction_avec_retard['NOMBRE DE CLIENTS PERDUS'] = prediction_avec_retard.apply(
    lambda x: get_number_of_lost_customer(x["RETARD"], x['NOMBRE DE PASSAGERS']), axis=1)

In [18]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,NOMBRE D'INDEMNITES DEMANDEES,INDEMNITES A PAYER,NOMBRE DE CLIENTS PERDUS
0,1627,Better Take A Train Airlines,HRE,1648,1328,5056000000,152092,1236,370800.0,49
1,510,Corporate Overlord Airways,BOM,1984,1576,40579000000,13527,1488,446400.0,59
2,53,Ne Va Pas Partout Airlines,TBS,1849,1548,6235000000,3432,0,0.0,0
3,901,Ne Va Pas Partout Airlines,BTS,1811,900,6235000000,75820,1358,407400.0,54
4,850,Neverland Airlines,SKT,1340,1687,7651000000,21866,1005,301500.0,40
...,...,...,...,...,...,...,...,...,...,...
95,1501,We Komen er Uiteindelijk Wel,KOJ,970,987,51000000000,47744,727,218100.0,29
96,718,Corporate Overlord Airways,NAP,1876,566,40579000000,63810,1407,422100.0,56
97,936,Ne Va Pas Partout Airlines,BHX,243,17,6235000000,64890,182,54600.0,7
98,1126,Neverland Airlines,CJJ,818,1583,7651000000,55850,613,183900.0,24


### Get the cost of all the lost client for the airlines

**Hypothèse** : 

On suppose qu'un client prend en moyenne 3 fois l'avion par an avec la même compagnie (on suppose une fidéité total des clients auprès de leur compagnie).

Donc si la compagnie perd un client, elle perd un cout de **3x"prix du billet"** par client

On suppose le prix du billet = 300€

In [19]:
def get_cost_of_lost_customer(nb_of_lost_customers, ticket_price=300, flight_frequency=3):
    return flight_frequency*ticket_price*nb_of_lost_customers

In [20]:
prediction_avec_retard['COUT DES CLIENTS PERDUS'] = prediction_avec_retard["NOMBRE DE CLIENTS PERDUS"].map(
                                                                        lambda x: get_cost_of_lost_customer(x))

In [21]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,NOMBRE D'INDEMNITES DEMANDEES,INDEMNITES A PAYER,NOMBRE DE CLIENTS PERDUS,COUT DES CLIENTS PERDUS
0,1627,Better Take A Train Airlines,HRE,1648,1328,5056000000,152092,1236,370800.0,49,44100
1,510,Corporate Overlord Airways,BOM,1984,1576,40579000000,13527,1488,446400.0,59,53100
2,53,Ne Va Pas Partout Airlines,TBS,1849,1548,6235000000,3432,0,0.0,0,0
3,901,Ne Va Pas Partout Airlines,BTS,1811,900,6235000000,75820,1358,407400.0,54,48600
4,850,Neverland Airlines,SKT,1340,1687,7651000000,21866,1005,301500.0,40,36000
...,...,...,...,...,...,...,...,...,...,...,...
95,1501,We Komen er Uiteindelijk Wel,KOJ,970,987,51000000000,47744,727,218100.0,29,26100
96,718,Corporate Overlord Airways,NAP,1876,566,40579000000,63810,1407,422100.0,56,50400
97,936,Ne Va Pas Partout Airlines,BHX,243,17,6235000000,64890,182,54600.0,7,6300
98,1126,Neverland Airlines,CJJ,818,1583,7651000000,55850,613,183900.0,24,21600


# TODO : 

- [x] Rajouter colonne avec pourcentage du chiffre d'affaire perdu 
- [x] Afficher nombre de vols en retard par compagnie 
- [x] combien de clients ils vont perdre : plus de 3h de retard --> taux d'attrition de 5% des clients
- [x] fréquence moyenne de réservation de vol avec la compagnie (3fs/an) : 5%x3x(prix du billet)
- [x] afficher le détail répartition des couts dans total à payer dans les graphes
- [ ] NEXT STEPS 

# BILAN : TOTAL A PAYER

In [23]:
cost_of_delay_gb_airlines = prediction_avec_retard[["RETARD", "COMPAGNIES","CHIFFRE D'AFFAIRE COMPAGNIE",
                                                    "COUT DU RETARD", "INDEMNITES A PAYER", "NOMBRE DE CLIENTS PERDUS", 
                                                    "COUT DES CLIENTS PERDUS"]]\
                            .groupby(['COMPAGNIES'], as_index=False)\
                            .agg({
                                "RETARD" : "count",
                                "CHIFFRE D'AFFAIRE COMPAGNIE":'first',
                                "COUT DU RETARD":'sum',
                                "INDEMNITES A PAYER":'sum',
                                "NOMBRE DE CLIENTS PERDUS": "sum", 
                                "COUT DES CLIENTS PERDUS": "sum"
                            }).rename(columns={"RETARD" : "NOMBRE DE RETARD"})

In [30]:
cost_of_delay_gb_airlines["TOTAL A PAYER"] = cost_of_delay_gb_airlines["COUT DU RETARD"]\
                                            + cost_of_delay_gb_airlines["INDEMNITES A PAYER"]\
                                            + cost_of_delay_gb_airlines["COUT DES CLIENTS PERDUS"]

In [31]:
cost_of_delay_gb_airlines["NV CHIFFRE D'AFFAIRE"] = cost_of_delay_gb_airlines["CHIFFRE D'AFFAIRE COMPAGNIE"]\
                                                    - cost_of_delay_gb_airlines["TOTAL A PAYER"]

In [32]:
cost_of_delay_gb_airlines["%CHIFFRE D'AFFAIRE LOST"] = \
(cost_of_delay_gb_airlines["TOTAL A PAYER"]/cost_of_delay_gb_airlines["CHIFFRE D'AFFAIRE COMPAGNIE"])*100

In [33]:
cost_of_delay_gb_airlines

,COMPAGNIES,NOMBRE DE RETARD,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,INDEMNITES A PAYER,NOMBRE DE CLIENTS PERDUS,COUT DES CLIENTS PERDUS,TOTAL A PAYER,NV CHIFFRE D'AFFAIRE,%CHIFFRE D'AFFAIRE LOST
0,Air Penguin,6,31064000000,304280,1992000.0,253,227700,2523980.0,3.106148e+10,0.008125
1,Air Piloter Sans Les Mains,9,7651000000,277310,2535600.0,334,300600,3113510.0,7.647886e+09,0.040694
2,Always A Problem Flights,9,51000000000,174530,2552850.0,320,288000,3015380.0,5.099698e+10,0.005913
3,Bel Air,6,3671000000,192102,1733550.0,225,202500,2128152.0,3.668872e+09,0.057972
4,Better Take A Train Airlines,5,5056000000,396646,1624800.0,214,192600,2214046.0,5.053786e+09,0.043790
5,Corporate Overlord Airways,9,40579000000,512223,3070800.0,401,360900,3943923.0,4.057506e+10,0.009719
6,Fliying Is Possible Inc.,7,2979000000,426581,1733500.0,229,206100,2366181.0,2.976634e+09,0.079429
7,IE 6.0 Flights,7,2660000000,182806,1388400.0,148,133200,1704406.0,2.658296e+09,0.064075
8,Morally Ambiguious Fligthts,8,6391000000,376346,2133000.0,281,252900,2762246.0,6.388238e+09,0.043221
9,Ne Va Pas Partout Airlines,6,6235000000,239191,1433100.0,189,170100,1842391.0,6.233158e+09,0.029549


In [34]:
fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "NV CHIFFRE D'AFFAIRE"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "TOTAL A PAYER"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

In [36]:
import plotly.graph_objects as go

for idx, company in enumerate(cost_of_delay_gb_airlines["COMPAGNIES"]):
    labels = ["NV CHIFFRE D'AFFAIRE","TOTAL A PAYER"]
    values = [cost_of_delay_gb_airlines.iloc[idx]["NV CHIFFRE D'AFFAIRE"],
              cost_of_delay_gb_airlines.iloc[idx]["TOTAL A PAYER"]]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
    fig.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig.update_layout(title_text=company)
    fig.show()